In [1]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
import datetime 
import time
import pytz
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
last_cats = cat_history_df[last_mask]['name'].unique()
this_cats = cat_history_df[this_mask]['name'].unique()

diff = [x for x in this_cats if x not in last_cats]
print(diff) 

### Testing Foster List Data Collection

In [ ]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

### Create old and New Files

In [ ]:
df = getFosterInfo()

In [ ]:
#make copy for edits
new_df = df.copy()

#changes to fields in existing rows
new_df.loc[3, 'cat_summary'] = 'test test test'
new_df.loc[7, 'cat_description'] = 'test test test'

#cat gets reserved
new_df.loc[9, 'cat_name'] = new_test.loc[9, 'cat_name'] + ' (reserved)'

#cat removed from foster list 
new_df.drop(12, inplace=True)

#cat added to list 
new_row = {
    'cat_id':['test-cat'],
    'cat_name':['test cat'],
    'cat_summary':['test'],
    'cat_timing':['test'],
    'cat_description':['test'],
    'cat_image':['test'],
    'cat_link':['test'],
    'import_at':[datetime.fromtimestamp(int(time.time()), tz=pytz.utc)]
}
new_row_df = pd.DataFrame(new_row)
new_df = pd.concat([new_df, new_row_df])


In [ ]:
new_df.to_csv('../output/foster_cats_test_new.csv', index=False)

### Compare Files

In [ ]:
old_df = pd.read_csv('../output/foster_cats_test_old.csv')
new_df = pd.read_csv('../output/foster_cats_test_new.csv')

In [ ]:
# merge dfs and compare
compare_df = pd.merge(old_df, new_df, how='outer', on='cat_id', suffixes=['_old','_new'])
loop_cols = [t for t in old_df.columns if 'cat_id' not in t and 'import_at' not in t]

for col in loop_cols:
    col_compare_df = compare_df[[c for c in compare_df.columns if col in c]]
    compare_df[f'was_{col}_modified'] = (col_compare_df[f'{col}_old'] != col_compare_df[f'{col}_new'])

# discard rows where nothing was changed
compare_df['was_modified'] = compare_df[[col for col in compare_df.columns \
                                        if 'was_' in col]].any(axis=1)
compare_df = compare_df[compare_df['was_modified'] == True]

# distinguish adds, deletions, modifications
def determine_change(old_val, new_val):
    if pd.isnull(old_val):
        return 'row_added'
    elif pd.isnull(new_val):
        return 'row_deleted'
    else:
        return 'row_modified'

compare_df['change_type'] = compare_df.apply(lambda x: determine_change(x['cat_name_old'], x['cat_name_new']), 
                                             axis=1)

In [ ]:
change_df = pd.read_csv('../output/foster_change_log.csv')

In [ ]:
if change_df[change_df['change_type'] == 'row_added'].shape[0] > 0:
    print('new cats added!')

In [ ]:
send_email(df, alert_message='change log test email', sender=my_username, password=my_password)

In [ ]:
from foster_cats import *

In [ ]:
# get last run + current run
old_fosters_df = pd.read_csv('../src/data/latest_fosters.csv')
new_fosters_df = getFosterInfo()

# split out html info from current run to avoid clutter in .csv
# we only need html for email alerts we might send 
new_fosters_write_df = new_fosters_df.drop(columns=['full_html'])

# save current run as history for next comparison
new_fosters_write_df.to_csv('../src/data/latest_fosters.csv', index=False)

# compare new and old runs to determine changes
changes_df = compare_snapshots(old_fosters_df, new_fosters_write_df)


if changes_df.shape[0] > 0:
    # logging.info('there have been changes since the last pipeline run')
    # logging.info('summary of changes: ')
    # logging.info(changes_df['change_type'].value_counts())
    
    # append changes to change history if it exists
    path = '../output/foster_change_log.csv'
    if os.path.exists(path):
        old_changes_df = pd.read_csv(path)
        changes_df = pd.concat([old_changes_df, changes_df])
    changes_df.to_csv(path, index=False)
    
    # if there were rows added, trigger alert

else:
    # logging.info('no changes since previous pipeline run')
    print('hi')

In [ ]:
recipients = pd.read_csv('../src/data/recipient_list.csv')

In [ ]:
recipient_list = ", ".join(recipients['email'].to_list())

In [ ]:
import smtplib
from email.mime.text import MIMEText
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')
recipients = pd.read_csv('../src/data/recipient_list.csv')
recipient_list = recipients['email'].to_list()

sender_email = my_username
sender_password = my_password

subject = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}: [TEST] New foster cat alert! {new_fosters_df.loc[0]['cat_summary']}"
body = f'''
    <html>
        <body>
        <h1> New cat alert! </h1>
            {new_fosters_df.loc[0]['full_html']}
        </body>
    </html>
    '''
html_message = MIMEText(body, 'html')
html_message['Subject'] = subject
html_message['From'] = sender_email
html_message['To'] = ", ".join(recipient_list)

server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(sender_email, sender_password)
server.sendmail(sender_email, recipient_list, html_message.as_string())
server.quit()

In [ ]:
new_fosters_df

In [ ]:
from configparser import ConfigParser
import ssl

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')

# Define the HTML document
body = '''
    <html>
        <body>
            <h1>Daily S&P 500 prices report</h1>
            <p>Hello, welcome to your report!</p>
        </body>
    </html>
    '''

# Generate today's date to be included in the email Subject
date_str = str(datetime.datetime.now())

# Create a MIMEMultipart class, and set up the From, To, Subject fields
email_message = MIMEMultipart()
email_message['From'] = my_username
email_message['To'] = my_username
email_message['Subject'] = f'Report email - {date_str}'

# Attach the html doc defined earlier, as a MIMEText html content type to the MIME message
email_message.attach(MIMEText(html, "html"))
# Convert it as a string
email_string = email_message.as_string()

# Connect to the Gmail SMTP server and Send Email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(my_username, my_password)
    server.sendmail(my_username, my_password, email_string)

# Appendix (Scratch)

In [ ]:
### first go to individual cat page and you can get the attributes via the below:

cafe_prospect = cat_soup.find('div', class_='comp-kjevammt2 YzqVVZ')


health_concerns = cat_soup.find('div', class_='comp-kjevamml YzqVVZ')


upcoming_appointments = cat_soup.find('div', class_='comp-kjevammp3 YzqVVZ')



## Create Change Log for Cafe Cats

In [78]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import pytz
import os
import logging

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
url = 'https://catcafebk.com/our-cats/?'
jobtime = datetime.fromtimestamp(int(time.time()), tz=pytz.utc)

# cat_df = get_latest_snapshot(url, jobtime)
# print('...retrieved latest cat data')

# #check if file exists and append
# if os.path.exists('cat_history.csv'):
#     print('...adding latest cat data to cat data history')
#     cat_history_df = pd.read_csv('cat_history.csv')
#     cat_history_df = pd.concat([cat_history_df, cat_df])
#     cat_history_df.to_csv('cat_history.csv', index=False)

# #otherwise create the file anew
# else:
#     cat_df.to_csv('cat_history.csv', index=False)

In [80]:
cat_history_df = pd.read_csv('../output/cat_history.csv')
cat_df = get_latest_snapshot(url=url, jobtime=datetime.fromtimestamp(int(time.time()), tz=pytz.utc))

Getting summary...
...done
Getting cat filter data...
Getting cafe_cat cats... 
Getting kid_approved cats... 
Getting dog_approved cats... 
Getting companion_cat cats... 
Getting bonded_pair cats... 
Getting single_cat cats... 
...done
Getting individual cat info (descriptions, type)...
...checking page for 101 cats...
...checked 20 of 101 cat pages...
...checked 40 of 101 cat pages...
...checked 60 of 101 cat pages...
...checked 80 of 101 cat pages...
...checked 100 of 101 cat pages...
...done


In [150]:
pwd

'/Users/lucasmazzotti/Documents/GitHub/cat-analytics/notebooks'

In [153]:
cat_history_df['system_import_at'].unique()

array(['2023-01-22 19:13:48+00:00', '2023-01-22 19:49:42+00:00',
       '2023-01-22 19:51:39+00:00', '2023-01-22 20:23:11+00:00',
       '2023-01-23 17:50:49+00:00', '2023-01-26 02:40:07+00:00',
       '2023-02-27 22:49:46+00:00'], dtype=object)

In [166]:
# last_cat_df = cat_history_df[cat_history_df['system_import_at'] == '2023-01-22 19:13:48+00:00']
# this_cat_df = cat_history_df[cat_history_df['system_import_at'] == '2023-02-27 22:49:46+00:00']

this_cat_df.to_csv('../src/data/latest_cafe_cats.csv', index=False)

In [168]:

last_cat_df = this_cat_df.copy()
url = 'https://catcafebk.com/our-cats/?'
jobtime = datetime.fromtimestamp(int(time.time()), tz=pytz.utc)

this_cat_df = get_latest_snapshot(url, jobtime)
print('...retrieved latest cat data')

Getting summary...
...done
Getting cat filter data...
Getting cafe_cat cats... 
Getting kid_approved cats... 
Getting dog_approved cats... 
Getting companion_cat cats... 
Getting bonded_pair cats... 
Getting single_cat cats... 
...done
Getting individual cat info (descriptions, type)...
...checking page for 106 cats...
...checked 20 of 106 cat pages...
...checked 40 of 106 cat pages...
...checked 60 of 106 cat pages...
...checked 80 of 106 cat pages...
...checked 100 of 106 cat pages...
...done
...retrieved latest cat data


In [171]:
this_cat_df['cat_id'] = this_cat_df['cat_id'].astype(int)

In [172]:
change_df = compare_snapshots(last_cat_df, this_cat_df, join_key='cat_id', exclude_cols=['system_import_at'])
# change_df.to_csv('../output/cafe_change_log.csv')

In [173]:
change_df.head()

,cat_id,url_old,cat_name_old,age_old,gender_old,cafe_cat_old,kid_approved_old,dog_approved_old,companion_cat_old,bonded_pair_old,single_cat_old,type_old,description_old,system_import_at_old,url_new,cat_name_new,age_new,gender_new,cafe_cat_new,kid_approved_new,dog_approved_new,companion_cat_new,bonded_pair_new,single_cat_new,type_new,description_new,system_import_at_new,was_url_modified,was_cat_name_modified,was_age_modified,was_gender_modified,was_cafe_cat_modified,was_kid_approved_modified,was_dog_approved_modified,was_companion_cat_modified,was_bonded_pair_modified,was_single_cat_modified,was_type_modified,was_description_modified,was_modified,change_type
0,18986799,https://catcafebk.com/our-cats/?cat=18986799,Alex the Pirate,2 Years 3 Months,Male,False,True,False,True,False,True,Domestic Short Hair,Alex the Pirate wants one thing only: all the ...,2023-02-27 22:49:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,True,True,True,True,True,True,True,True,True,True,True,True,row_deleted
1,18949086,https://catcafebk.com/our-cats/?cat=18949086,Bada & Pow,5 Months 24 Days,Male,False,True,True,False,True,False,Domestic Short Hair,"Bada and Pow are two adorable, playful brother...",2023-02-27 22:49:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,True,True,True,True,True,True,True,True,True,True,True,True,row_deleted
2,19015693,https://catcafebk.com/our-cats/?cat=19015693,Bella & Clara,7 Months 14 Days,Female,False,False,False,False,True,False,Domestic Short Hair,Bella and Clara are the sweetest bonded sister...,2023-02-27 22:49:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,True,True,True,True,True,True,True,True,True,True,True,True,row_deleted
3,19048828,https://catcafebk.com/our-cats/?cat=19048828,Benjiro,1 Year 1 Month,Male,False,False,False,False,False,False,Domestic Short Hair,Benjiro is a sweet and relaxed black panther w...,2023-02-27 22:49:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,True,True,True,True,True,True,True,True,True,True,True,True,row_deleted
4,19014973,https://catcafebk.com/our-cats/?cat=19014973,Bing & Bang,5 Months 5 Days,NaN,True,True,True,False,True,False,Domestic Short Hair,This gorgeous brother and sister love each oth...,2023-02-27 22:49:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,True,True,False,True,True,True,True,True,True,True,True,True,row_deleted


In [157]:
from utilities import create_field_history

In [174]:
df = create_field_history(change_df, id_col='cat_id')

In [179]:
last_df = pd.read_csv('../output/cafe_field_history.csv')

In [176]:
df.head()

,cat_id,field,value,valid_starting_at,valid_ending_at
106,18578528,age,1 Year 2 Months,2023-07-08 16:37:13+00:00,NaT
260,18259464,age,1 Year 4 Months,2023-07-08 16:37:13+00:00,NaT
261,18259464,description,Dipsy and Pinky Winky are playful balls of ene...,2023-07-08 16:37:13+00:00,NaT
274,19014938,age,3 Years 11 Months,2023-07-08 16:37:13+00:00,NaT
275,19014938,kid_approved,False,2023-07-08 16:37:13+00:00,NaT


In [180]:
last_df.head()

,cat_id,field,value,valid_starting_at,valid_ending_at
0,18949086,age,5 Months 24 Days,2023-07-08 16:27:17+00:00,NaN
1,18948728,age,7 Months 5 Days,2023-07-08 16:27:17+00:00,NaN
2,18948728,description,Popeye and Bluto are a beautiful bonded siblin...,2023-07-08 16:27:17+00:00,NaN
3,17834315,age,2 Years 11 Months,2023-07-08 16:27:17+00:00,NaN
4,17834315,kid_approved,True,2023-07-08 16:27:17+00:00,NaN


In [198]:
def update_field_history(last_field_history_df:pd.DataFrame, this_field_history_df:pd.DataFrame, id_col:str) -> pd.DataFrame:
    '''
    basic function to combine new and old field history labels to update valid_starting_at and valid_ending_at timestamps where we have
    multiple records corresponding to a given id,field,value combination
    '''
    df = pd.merge(left=last_field_history_df, right=this_field_history_df, how='outer', on=[id_col,'field','value'])
    df['valid_starting_at'] = np.where(df['valid_starting_at_x'].isnull(), df['valid_starting_at_y'], df['valid_starting_at_x'])
    df['valid_ending_at'] = np.where(df['valid_ending_at_x'].isnull(), df['valid_ending_at_y'], df['valid_ending_at_x'])

    df.drop(columns=['valid_starting_at_x','valid_starting_at_y','valid_ending_at_x','valid_ending_at_y'], inplace=True)
    
    return df


/var/folders/_f/vck1fqj55xqftl_63ylyj_sw0000gp/T/ipykernel_14459/4293787365.py:3: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  test['valid_ending_at'] = np.where(test['valid_ending_at_x'].isnull(), test['valid_ending_at_y'], test['valid_ending_at_x'])


,cat_id,field,value,valid_starting_at,valid_ending_at
0,18949086,age,5 Months 24 Days,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
1,18948728,age,7 Months 5 Days,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
2,18948728,description,Popeye and Bluto are a beautiful bonded siblin...,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
3,17834315,age,2 Years 11 Months,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
4,17834315,kid_approved,True,2023-07-08 16:27:17+00:00,NaT
...,...,...,...,...,...
3100,18587232,companion_cat,False,NaN,2023-07-08 16:37:13+00:00
3101,18587232,bonded_pair,True,NaN,2023-07-08 16:37:13+00:00
3102,18587232,single_cat,False,NaN,2023-07-08 16:37:13+00:00
3103,18587232,type,Domestic Short Hair,NaN,2023-07-08 16:37:13+00:00


In [200]:
test[test['cat_id'] == 18948728]

,cat_id,field,value,valid_starting_at,valid_ending_at
1,18948728,age,7 Months 5 Days,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
2,18948728,description,Popeye and Bluto are a beautiful bonded siblin...,2023-07-08 16:27:17+00:00,2023-07-08 16:37:13+00:00
794,18948728,age,6 Months 0 Days,NaT,2023-07-08 16:27:17+00:00
795,18948728,description,"Popeye and Bluto are a very sweet, but shy pai...",NaT,2023-07-08 16:27:17+00:00
2340,18948728,url,https://catcafebk.com/our-cats/?cat=18948728,NaT,2023-07-08 16:37:13+00:00
2341,18948728,cat_name,Bluto & Popeye,NaT,2023-07-08 16:37:13+00:00
2342,18948728,cafe_cat,False,NaT,2023-07-08 16:37:13+00:00
2343,18948728,kid_approved,False,NaT,2023-07-08 16:37:13+00:00
2344,18948728,dog_approved,False,NaT,2023-07-08 16:37:13+00:00
2345,18948728,companion_cat,False,NaT,2023-07-08 16:37:13+00:00


In [185]:
test.shape

(3105, 7)

In [194]:
last_df_copy = last_df.replace(pd.NaT, np.nan)
this_df_copy = df.replace(pd.NaT, np.nan)

test3 = last_df_copy.combine_first(this_df_copy)

/var/folders/_f/vck1fqj55xqftl_63ylyj_sw0000gp/T/ipykernel_14459/3652006496.py:4: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  test3 = last_df_copy.combine_first(this_df_copy)


In [196]:
test3[(test3['cat_id'] == 18949086) & (test3['field'] == 'age')]

,cat_id,field,value,valid_starting_at,valid_ending_at
0,18949086,age,5 Months 24 Days,2023-07-08 16:27:17+00:00,NaT
793,18949086,age,4 Months 19 Days,NaT,2023-07-08 16:27:17+00:00
2043,18949086,age,5 Months 24 Days,NaT,2023-07-08 16:37:13+00:00


In [186]:
test2 = last_df.combine_first(df)
test2.shape

/var/folders/_f/vck1fqj55xqftl_63ylyj_sw0000gp/T/ipykernel_14459/1891787671.py:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, UTC])
  test2 = last_df.combine_first(df)


(3188, 5)

In [193]:
type(test2[(test2['cat_id'] == 18949086) & (test2['field'] == 'age')].loc[0]['valid_ending_at'])

pandas._libs.tslibs.nattype.NaTType

In [145]:
# row modified
df[df['cat_id'] == 18844861]

,cat_id,field,value,valid_starting_at,valid_ending_at
329,18844861,age,9 Months 3 Days,2023-07-08 15:45:35+00:00,NaT
330,18844861,description,This sweet duo with different personalities ha...,2023-07-08 15:45:35+00:00,NaT
2437,18844861,age,3 Months 21 Days,NaT,2023-07-08 15:45:35+00:00
2438,18844861,description,"Though these two bundles of cuddles, purrs and...",NaT,2023-07-08 15:45:35+00:00


In [146]:
# row deleted
df[df['cat_id'] == 18576198]

,cat_id,field,value,valid_starting_at,valid_ending_at
2108,18576198,url,https://catcafebk.com/our-cats/?cat=18576198,NaT,2023-07-08 15:45:35+00:00
2109,18576198,cat_name,Aeon,NaT,2023-07-08 15:45:35+00:00
2110,18576198,age,10 Years 5 Months,NaT,2023-07-08 15:45:35+00:00
2111,18576198,gender,Female,NaT,2023-07-08 15:45:35+00:00
2112,18576198,cafe_cat,False,NaT,2023-07-08 15:45:35+00:00
2113,18576198,kid_approved,False,NaT,2023-07-08 15:45:35+00:00
2114,18576198,dog_approved,False,NaT,2023-07-08 15:45:35+00:00
2115,18576198,companion_cat,True,NaT,2023-07-08 15:45:35+00:00
2116,18576198,bonded_pair,False,NaT,2023-07-08 15:45:35+00:00
2117,18576198,single_cat,True,NaT,2023-07-08 15:45:35+00:00


In [147]:
# row added
df[df['cat_id'] == 19381074]

,cat_id,field,value,valid_starting_at,valid_ending_at
1158,19381074,url,https://catcafebk.com/our-cats/?cat=19381074,2023-07-08 15:45:35+00:00,NaT
1159,19381074,cat_name,Adventureland & Epcot,2023-07-08 15:45:35+00:00,NaT
1160,19381074,age,2 Months 27 Days,2023-07-08 15:45:35+00:00,NaT
1161,19381074,cafe_cat,False,2023-07-08 15:45:35+00:00,NaT
1162,19381074,kid_approved,False,2023-07-08 15:45:35+00:00,NaT
1163,19381074,dog_approved,False,2023-07-08 15:45:35+00:00,NaT
1164,19381074,companion_cat,False,2023-07-08 15:45:35+00:00,NaT
1165,19381074,bonded_pair,True,2023-07-08 15:45:35+00:00,NaT
1166,19381074,single_cat,False,2023-07-08 15:45:35+00:00,NaT
1167,19381074,type,Domestic Short Hair,2023-07-08 15:45:35+00:00,NaT


In [126]:
df

,cat_id,field,value_x,valid_ending_at,value_y,valid_starting_at
0,18844861,age,3 Months 21 Days,2023-07-04 23:24:13+00:00,9 Months 3 Days,2023-07-04 23:24:13+00:00
1,18844861,description,"Though these two bundles of cuddles, purrs and...",2023-07-04 23:24:13+00:00,This sweet duo with different personalities ha...,2023-07-04 23:24:13+00:00
2,17850952,age,1 Year 4 Months,2023-07-04 23:24:13+00:00,1 Year 9 Months,2023-07-04 23:24:13+00:00
3,17850952,description,Zapps and Funyun are a playful and ridiculous ...,2023-07-04 23:24:13+00:00,Funyun and Zappa are ridiculously rambunctious...,2023-07-04 23:24:13+00:00
4,18542746,age,3 Years 7 Months,2023-07-04 23:24:13+00:00,4 Years 1 Month,2023-07-04 23:24:13+00:00
5,18542746,description,Heater Howard is a sweet cat who loves playing...,2023-07-04 23:24:13+00:00,Heater Howard is a sweet orange lady who loves...,2023-07-04 23:24:13+00:00
6,18950721,age,3 Months 20 Days,2023-07-04 23:24:13+00:00,9 Months 2 Days,2023-07-04 23:24:13+00:00
7,18950721,cafe_cat,False,2023-07-04 23:24:13+00:00,True,2023-07-04 23:24:13+00:00
8,18892061,age,7 Years 6 Months,2023-07-04 23:24:13+00:00,7 Years 11 Months,2023-07-04 23:24:13+00:00
9,18724899,age,8 Months 7 Days,2023-07-04 23:24:13+00:00,1 Year 1 Month,2023-07-04 23:24:13+00:00


In [87]:
rows_modified_df.tail(1)

,cat_id,url_old,cat_name_old,age_old,gender_old,cafe_cat_old,kid_approved_old,dog_approved_old,companion_cat_old,bonded_pair_old,single_cat_old,type_old,description_old,system_import_at_old,url_new,cat_name_new,age_new,gender_new,cafe_cat_new,kid_approved_new,dog_approved_new,companion_cat_new,bonded_pair_new,single_cat_new,type_new,description_new,system_import_at_new,was_url_modified,was_cat_name_modified,was_age_modified,was_gender_modified,was_cafe_cat_modified,was_kid_approved_modified,was_dog_approved_modified,was_companion_cat_modified,was_bonded_pair_modified,was_single_cat_modified,was_type_modified,was_description_modified,was_modified,change_type
115,18587232,https://catcafebk.com/our-cats/?cat=18587232,Wonky & Wheezy,9 Months 17 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Wonky may be the cuddliest kitten in the world...,2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18587232,Wonky & Wheezy,1 Year 2 Months,NaN,False,True,True,False,True,False,Domestic Short Hair,Wonky may be the cuddliest kitten in the world...,2023-07-04 21:45:59+00:00,False,False,True,True,False,False,False,False,False,False,False,False,True,row_modified


In [62]:
rows_deleted_df

,cat_id,field,value,valid_ending_at
0,18576198,url,https://catcafebk.com/our-cats/?cat=18576198,2023-07-04 21:36:02+00:00
1,18949108,url,https://catcafebk.com/our-cats/?cat=18949108,2023-07-04 21:36:02+00:00
2,18049093,url,https://catcafebk.com/our-cats/?cat=18049093,2023-07-04 21:36:02+00:00
3,18844592,url,https://catcafebk.com/our-cats/?cat=18844592,2023-07-04 21:36:02+00:00
4,18949086,url,https://catcafebk.com/our-cats/?cat=18949086,2023-07-04 21:36:02+00:00
...,...,...,...,...
1159,18667410,description,Cat? Rabbit? Muppet? Waldorf is a senior cat w...,2023-07-04 21:36:02+00:00
1160,18568223,description,"Where and Wal aren’t siblings, but they were c...",2023-07-04 21:36:02+00:00
1161,18634866,description,Will and Way are paws down the most affectiona...,2023-07-04 21:36:02+00:00
1162,18895748,description,Winston is the charmer of the duo. She’s more ...,2023-07-04 21:36:02+00:00


In [53]:
rows_added_df.head()

,cat_id,field,value,valid_starting_at
0,19381074,url,https://catcafebk.com/our-cats/?cat=19381074,2023-07-04 16:36:23
1,19452563,url,https://catcafebk.com/our-cats/?cat=19452563,2023-07-04 16:36:23
2,17351058,url,https://catcafebk.com/our-cats/?cat=17351058,2023-07-04 16:36:23
3,19130200,url,https://catcafebk.com/our-cats/?cat=19130200,2023-07-04 16:36:23
4,19453314,url,https://catcafebk.com/our-cats/?cat=19453314,2023-07-04 16:36:23


In [49]:
rows_added_df[rows_added_df['cat_id'] == 17351058]

,cat_id,variable,value,valid_starting_at
2,17351058,url_new,https://catcafebk.com/our-cats/?cat=17351058,2023-07-04 16:36:23
83,17351058,cat_name_new,Angelo,2023-07-04 16:36:23
164,17351058,age_new,6 Years 8 Months,2023-07-04 16:36:23
245,17351058,gender_new,Male,2023-07-04 16:36:23
326,17351058,cafe_cat_new,True,2023-07-04 16:36:23
407,17351058,kid_approved_new,True,2023-07-04 16:36:23
488,17351058,dog_approved_new,True,2023-07-04 16:36:23
569,17351058,companion_cat_new,True,2023-07-04 16:36:23
650,17351058,bonded_pair_new,False,2023-07-04 16:36:23
731,17351058,single_cat_new,True,2023-07-04 16:36:23


In [18]:
change_df[change_df['change_type'] == 'row_modified']

,cat_id,url_old,cat_name_old,age_old,gender_old,cafe_cat_old,kid_approved_old,dog_approved_old,companion_cat_old,bonded_pair_old,single_cat_old,type_old,description_old,system_import_at_old,url_new,cat_name_new,age_new,gender_new,cafe_cat_new,kid_approved_new,dog_approved_new,companion_cat_new,bonded_pair_new,single_cat_new,type_new,description_new,system_import_at_new,was_url_modified,was_cat_name_modified,was_age_modified,was_gender_modified,was_cafe_cat_modified,was_kid_approved_modified,was_dog_approved_modified,was_companion_cat_modified,was_bonded_pair_modified,was_single_cat_modified,was_type_modified,was_description_modified,was_modified,change_type
28,18844861,https://catcafebk.com/our-cats/?cat=18844861,Fennel & Stowaway,3 Months 21 Days,Male,False,True,True,False,True,False,Domestic Short Hair,"Though these two bundles of cuddles, purrs and...",2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18844861,Fennel & Stowaway,9 Months 3 Days,Male,False,True,True,False,True,False,Domestic Short Hair,This sweet duo with different personalities ha...,2023-07-04 16:36:23+00:00,False,False,True,False,False,False,False,False,False,False,False,True,True,row_modified
31,17850952,https://catcafebk.com/our-cats/?cat=17850952,Funyun & Zapps,1 Year 4 Months,Female,False,False,False,False,True,False,Domestic Short Hair,Zapps and Funyun are a playful and ridiculous ...,2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=17850952,Funyun & Zapps,1 Year 9 Months,Female,False,False,False,False,True,False,Domestic Short Hair,Funyun and Zappa are ridiculously rambunctious...,2023-07-04 16:36:23+00:00,False,False,True,False,False,False,False,False,False,False,False,True,True,row_modified
38,18542746,https://catcafebk.com/our-cats/?cat=18542746,Heater Howard,3 Years 7 Months,Female,False,False,False,True,False,True,Domestic Short Hair,Heater Howard is a sweet cat who loves playing...,2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18542746,Heater Howard,4 Years 1 Month,Female,False,False,False,True,False,True,Domestic Short Hair,Heater Howard is a sweet orange lady who loves...,2023-07-04 16:36:23+00:00,False,False,True,False,False,False,False,False,False,False,False,True,True,row_modified
41,18950721,https://catcafebk.com/our-cats/?cat=18950721,Hurricane & Tsunami,3 Months 20 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Hurricane is a super brave and friendly little...,2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18950721,Hurricane & Tsunami,9 Months 2 Days,,True,True,True,False,True,False,Domestic Short Hair,Hurricane is a super brave and friendly little...,2023-07-04 16:36:23+00:00,False,False,True,True,True,False,False,False,False,False,False,False,True,row_modified
42,18892061,https://catcafebk.com/our-cats/?cat=18892061,Jack & Janet,7 Years 6 Months,NaN,False,False,False,False,True,False,Domestic Short Hair,Jack and Janet are the cutest tuxedo cats that...,2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18892061,Jack & Janet,7 Years 11 Months,,False,False,False,False,True,False,Domestic Short Hair,Jack and Janet are the cutest tuxedo cats that...,2023-07-04 16:36:23+00:00,False,False,True,True,False,False,False,False,False,False,False,False,True,row_modified
43,18724899,https://catcafebk.com/our-cats/?cat=18724899,Jackson Galaxy & Liberty,8 Months 7 Days,Male,False,True,True,False,True,False,Domestic Short Hair,"Liberty can be a bit shy at first, but energet...",2023-01-22 19:13:48+00:00,https://catcafebk.com/our-cats/?cat=18724899,Jackson Galaxy & Liberty,1 Year 1 Month,Male,False,True,True,False,True,False,Domestic Short Hair,"Liberty can be a bit shy at first, but energet...",2023-07-04 16:36:23+00:00,False,False,True,False,False,False,False,False,False,False,False,False,True,row_modified
52,18844560,https://catcafebk.com/our-cats/?cat=18844560,Lucky Penny & Flour Sack,6 Months 8 Days,NaN,False,True,True,False,True,False,Domestic Short Hair,Lucky Penny

In [24]:
for col in [col for col in change_df.columns if '_old' in col]:
    

url_old
cat_name_old
age_old
gender_old
cafe_cat_old
kid_approved_old
dog_approved_old
companion_cat_old
bonded_pair_old
single_cat_old
type_old
description_old
system_import_at_old


## Script Debugging

In [1]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import pytz
import os
import logging

In [46]:
# time script 
begin_time = time.perf_counter()

# get last run + current run
old_fosters_df = pd.read_csv('../src/data/latest_fosters.csv')
try:
    new_fosters_df = getFosterInfo()
except Exception as e:
    logging.info(f'error getting new info: {e}')
    logging.info(e)
    
    


In [90]:
soup = get_cat_page('https://bbawcfosterteam.wixsite.com/cats/availablecats')
cats = soup.find('fluid-columns-repeater').find_all('div', class_ = 'comp-kjevao4x5 YzqVVZ wixui-repeater__item')
jobtime = datetime.fromtimestamp(int(time.time()), tz=pytz.utc)

results_list = []

for cat in cats:
    
    # get data 
    text_data = cat.find('div', class_='hFQZVn comp-kjevao571')
    cat_image = cat.find('img').attrs['src']
    cat_link = cat.find('a').attrs['href']
    
    # get basic info from text
    cat_name = text_data.find('h4').text
    cat_summary = text_data.find(id=re.compile('comp-kjevl5nv')).text
    cat_timing = text_data.find(id=re.compile('comp-kjevrpz5')).text
    cat_description = text_data.find(id=re.compile('comp-kjevao593')).text
    
    # construct an ID for cat (sometimes the name changes to "Name (Reserved)" or "Name (Hold)")
    # this will make it easier to compare snapshots
    cat_id = cat_name.replace(' (reserved)', '').lower()\
            .replace(' (hold)', '')\
            .replace(' ', '-')

    
    results_list.append(
        {'cat_id':cat_id,
        'cat_name':cat_name,
        'cat_summary':cat_summary,
        'cat_timing':cat_timing,
        'cat_description':cat_description,
        'cat_image':cat_image,
        'cat_link':cat_link,
        'full_html':cat}
    )
df = pd.json_normalize(results_list)
df['import_at'] = datetime.fromtimestamp(int(time.time()), tz=pytz.utc)

In [92]:
df

,cat_id,cat_name,cat_summary,cat_timing,cat_description,cat_image,cat_link,full_html,import_at
0,bandito,Bandito (reserved),2 year-old black and white male,"Ready now, at the cafe",From his rescuer: Bandito was found in Bensonh...,https://static.wixstatic.com/media/7f467d_02aa...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
1,buffy,Buffy,2ish year old white and tabby lady,"Ready now, fosterer can meet you at the cafe",Buffy was found in a basement along with her 3...,https://static.wixstatic.com/media/7f467d_4d1a...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
2,castle,Castle,8 year-old tabby lady,Needs a home ASAP,From her fosterer: Castle is the sweetest frie...,https://static.wixstatic.com/media/7f467d_cd0d...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
3,chumbo-and-micheal,Chumbo and Micheal,5 year-old grey and orange tabby friends,"Ready starting 4/3, at the cafe",Chumbo and Micheal have been living outside to...,https://static.wixstatic.com/media/7f467d_50d7...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
4,dandelion-and-marshmallow,Dandelion and Marshmallow,6 month-old long haired siblings,"Ready now, fosterer can meet you at the cafe","If you like your kittens with extra fluff, it'...",https://static.wixstatic.com/media/7f467d_5f44...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
5,earthquake-and-wildfire-(reserved),Earthquake and Wildfire (Reserved),5 month-old siblings,Need a new foster by 4/8,Say hello to Earthquake and Firestorm! This bo...,https://static.wixstatic.com/media/7f467d_2767...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
6,ensign-ro-and-the-maquis,Ensign Ro and The Maquis (hold),Young adult lady,"Ready evening of 3/24, at the cafe",This sweet friend appeared near a feeding stat...,https://static.wixstatic.com/media/7f467d_d8ab...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
7,florence-&-percy,Florence & Percy,8 month and 2 year old buddies,"Ready now, at the cafe",Percy and Florence are affectionate chill budd...,https://static.wixstatic.com/media/7f467d_5da0...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
8,hijiki,Hijiki (reserved),Adult all-black male,"Ready now, at the cafe",Hijiki is a gentle cat who appreciates pets an...,https://static.wixstatic.com/media/7f467d_1026...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00
9,jeff-mcneil,Jeff McNeil,2 year-old orange and white male,"Ready now, at the cafe",My best friend! Jeff appeared in my back yard ...,https://static.wixstatic.com/media/7f467d_a057...,https://bbawcfosterteam.wixsite.com/cats/avali...,"[[[], []], [[<div class=""MazNVa comp-kjevao535...",2023-04-09 00:02:53+00:00


In [88]:
text_data.find_all('div')

[<div class="nTiihL wixui-box" data-testid="container-bg"></div>,
 <div class="" data-mesh-id="comp-kjevao571__efb6188a-f785-474e-ae77-bed10011a4adinlineContent" data-testid="inline-content"><div data-mesh-id="comp-kjevao571__efb6188a-f785-474e-ae77-bed10011a4adinlineContent-gridContainer" data-testid="mesh-container-content"><div class="BaOVQ8 tz5f0K comp-kjevao58 wixui-rich-text" data-testid="richTextElement" id="comp-kjevao58__efb6188a-f785-474e-ae77-bed10011a4ad"><h4 class="font_4" style="font-size:40px;">Buffy</h4></div><div class="BaOVQ8 tz5f0K comp-kjevl5nv wixui-rich-text" data-testid="richTextElement" id="comp-kjevl5nv__efb6188a-f785-474e-ae77-bed10011a4ad"><p class="font_7" style="font-size:17px;">2ish year old white and tabby lady</p></div><div class="BaOVQ8 tz5f0K comp-kjevrpz5 wixui-rich-text" data-testid="richTextElement" id="comp-kjevrpz5__efb6188a-f785-474e-ae77-bed10011a4ad"><p class="font_7" style="font-size:17px;">Ready now, fosterer can meet you at the cafe</p></div

In [59]:
text_data.find_all('div')[4].__dict__

{'parser_class': bs4.BeautifulSoup,
 'name': 'div',
 'namespace': None,
 '_namespaces': {},
 'prefix': None,
 'sourceline': 669,
 'sourcepos': 11630,
 'known_xml': False,
 'attrs': {'id': 'comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38',
  'class': ['BaOVQ8', 'tz5f0K', 'comp-kjevl5nv', 'wixui-rich-text'],
  'data-testid': 'richTextElement'},
 'contents': [<p class="font_7" style="font-size:17px;">2 year-old tabby male</p>],
 'parent': <div data-mesh-id="comp-kjevao571__1f305ad1-3b79-407b-828e-73da46af0d38inlineContent-gridContainer" data-testid="mesh-container-content"><div class="BaOVQ8 tz5f0K comp-kjevao58 wixui-rich-text" data-testid="richTextElement" id="comp-kjevao58__1f305ad1-3b79-407b-828e-73da46af0d38"><h4 class="font_4" style="font-size:40px;">Whizbang (reserved)</h4></div><div class="BaOVQ8 tz5f0K comp-kjevl5nv wixui-rich-text" data-testid="richTextElement" id="comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38"><p class="font_7" style="font-size:17px;">2 year-old tabby

In [64]:
text_data.find('attrs=')

In [84]:
text_data.find(id='comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38')

<div class="BaOVQ8 tz5f0K comp-kjevl5nv wixui-rich-text" data-testid="richTextElement" id="comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38"><p class="font_7" style="font-size:17px;">2 year-old tabby male</p></div>

In [83]:
text_data.find('div', {'id': 'comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38',
  'class': ['BaOVQ8', 'tz5f0K', 'comp-kjevl5nv', 'wixui-rich-text'],
  'data-testid': 'richTextElement'}).text

'2 year-old tabby male'

In [81]:
text_data.find_all('div')[4].__dict__

{'parser_class': bs4.BeautifulSoup,
 'name': 'div',
 'namespace': None,
 '_namespaces': {},
 'prefix': None,
 'sourceline': 669,
 'sourcepos': 11630,
 'known_xml': False,
 'attrs': {'id': 'comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38',
  'class': ['BaOVQ8', 'tz5f0K', 'comp-kjevl5nv', 'wixui-rich-text'],
  'data-testid': 'richTextElement'},
 'contents': [<p class="font_7" style="font-size:17px;">2 year-old tabby male</p>],
 'parent': <div data-mesh-id="comp-kjevao571__1f305ad1-3b79-407b-828e-73da46af0d38inlineContent-gridContainer" data-testid="mesh-container-content"><div class="BaOVQ8 tz5f0K comp-kjevao58 wixui-rich-text" data-testid="richTextElement" id="comp-kjevao58__1f305ad1-3b79-407b-828e-73da46af0d38"><h4 class="font_4" style="font-size:40px;">Whizbang (reserved)</h4></div><div class="BaOVQ8 tz5f0K comp-kjevl5nv wixui-rich-text" data-testid="richTextElement" id="comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38"><p class="font_7" style="font-size:17px;">2 year-old tabby

In [71]:
text_data.find("div",{"id":"comp-kjevao593__ab9a692b-f81e-4d36-876a-129cf08709d2"})

In [66]:
text_data.find_all('div')[4].__dict__

{'parser_class': bs4.BeautifulSoup,
 'name': 'div',
 'namespace': None,
 '_namespaces': {},
 'prefix': None,
 'sourceline': 669,
 'sourcepos': 11630,
 'known_xml': False,
 'attrs': {'id': 'comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38',
  'class': ['BaOVQ8', 'tz5f0K', 'comp-kjevl5nv', 'wixui-rich-text'],
  'data-testid': 'richTextElement'},
 'contents': [<p class="font_7" style="font-size:17px;">2 year-old tabby male</p>],
 'parent': <div data-mesh-id="comp-kjevao571__1f305ad1-3b79-407b-828e-73da46af0d38inlineContent-gridContainer" data-testid="mesh-container-content"><div class="BaOVQ8 tz5f0K comp-kjevao58 wixui-rich-text" data-testid="richTextElement" id="comp-kjevao58__1f305ad1-3b79-407b-828e-73da46af0d38"><h4 class="font_4" style="font-size:40px;">Whizbang (reserved)</h4></div><div class="BaOVQ8 tz5f0K comp-kjevl5nv wixui-rich-text" data-testid="richTextElement" id="comp-kjevl5nv__1f305ad1-3b79-407b-828e-73da46af0d38"><p class="font_7" style="font-size:17px;">2 year-old tabby

In [13]:
# get data 
text_data = cat.find('div', class_='VgO9Yg')
cat_image = cat.find('img').attrs['src']
cat_link = cat.find('a').attrs['href']

# get basic info from text
cat_name = text_data.find('h4').text
cat_summary = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevl5nv wixui-text").text
cat_timing = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevrpz5 wixui-text").text
cat_description = text_data.find('div', class_="BaOVQ8 tz5f0K comp-kjevao593 wixui-text").text


AttributeError: 'NoneType' object has no attribute 'text'

In [20]:
cat_link

'https://bbawcfosterteam.wixsite.com/cats/avaliblecats/bandito-(reserved)'

In [36]:
df.to_csv('../src/data/latest_fosters.csv', index=False)

In [ ]:
new_fosters_df.to_csv('data/latest_fosters.csv', index=False)

# compare new and old runs to determine changes
current_changes_df = compare_snapshots(old_fosters_df, new_fosters_df)

if current_changes_df.shape[0] > 0:
    logging.info('there have been changes since the last pipeline run')
    logging.info('summary of changes: ')
    logging.info(current_changes_df['change_type'].value_counts())
    
    # append changes to change history if it exists
    path = '../output/foster_change_log.csv'
    if os.path.exists(path):
        logging.info('appending to existing change log')
        old_changes_df = pd.read_csv(path)
        changes_df = pd.concat([old_changes_df, current_changes_df])
        changes_df.to_csv(path, index=False)
    else: 
        logging.info('no existing change log found, creating a new one')
        current_changes_df.to_csv(path, index=False)
    
    # if there were rows added, trigger alert
    logging.info('checking for situation: new cat added')
    mask = (current_changes_df['change_type'] == 'row_added')
    if current_changes_df[mask].shape[0] > 0:
        
        logging.info('new cats were added. preparing to send email alerts')
        
        # prepare email creds 
        parser = ConfigParser()
        _ = parser.read('../credentials.cfg')
        sender = parser.get('my_email','my_username')
        password = parser.get('my_email','my_password')
        
        # get recipient list for email
        recipients_df = pd.read_csv('data/recipient_list.csv')
        recipient_list = recipients_df['email'].to_list()
        
        # loop through added cats and send emails
        for cat_id in current_changes_df[mask]['cat_id'].unique():
            cat_email_df = new_fosters_df[new_fosters_df['cat_id'] == cat_id]
            send_new_foster_email(cat_email_df, sender, recipient_list, password)
            logging.info(f"sent email for new cat: {cat_id}")
else:
    logging.info('no changes since previous pipeline run')

end_time = time.perf_counter()
total_time = end_time - begin_time
logging.info(f'script completed in {total_time:0.6f} seconds')